## Codewars Grader
Vitor Gonçalves
vgoncalves@usn.org

---

> Note: Need to add ability to take in CSV files of the student's usernames </br>
> Note: Need to add ability to grade certain codewars id's

procedure:
1. get list of students usernames in a CSV file
2. get list of codewars ids assigned in a CSV file, id, name, number of points
3. create a function that can grade a single question to each user
4. add pandas columns to those questions, either a true or false from the function
5. create a function that reads each test question column and grades it in a new column
6. add input to that function that only shows the grade or the 

---
link: https://dev.codewars.com/#get-user

In [ ]:
import requests
import pandas as pd
import time

## IMPORT CSV

link: https://sparkbyexamples.com/pandas/pandas-read-excel-multiple-sheets-in-pandas/

## QUESTION SUMMARY FUNCTIONS

In [ ]:
def getKataResponse(kata_id):
    #INPUT: string of the question id
    #OUTPUT: Total JSON of the challenge question
    response = requests.get(f"https://www.codewars.com/api/v1/code-challenges/{kata_id}")
    return response

In [ ]:
def kataDetails(kata_id):
    #INPUT: json of the kata info
    #OUTPUT: Pandas Dataframe containing name, slug, rank, url
    json = getKataResponse(kata_id).json()
    
    name = json["name"]
    slug = json["slug"]
    rank = json["rank"]["name"]
    url = json["url"]
    
    return pd.DataFrame({
        "Name":[name],
        "Slug":[slug],
        "Rank":[rank],
        "URL":[url],
    })

## USER SUMMARY FUNCTIONS

In [ ]:
def getSummaryResponse(user):
    #INPUT: string of the student's username
    #OUTPUT: API Response from the codewars website, user's summary
    
    response = requests.get(f"https://www.codewars.com/api/v1/users/{user}")
    return response

In [ ]:
def readUserSummary(response):
    #INPUT: Https response from codewars
    #OUTPUT: PANDAS Dataframe containing the information we can about
    
    #creates the required columns from codewars
    name = response.json()["name"]
    user = response.json()["username"]
    pyrank = response.json()['ranks']["languages"]["python"]["rank"]
    pyscore = response.json()['ranks']["languages"]["python"]["score"]
    completed = response.json()['codeChallenges']['totalCompleted']
    
    return pd.DataFrame({"Name": [name],
                         "User": [user],
                        "Rank": [pyrank],
                        "Score": [pyscore],
                        "Completed": [completed]})

In [ ]:
def printClassSummary(list_of_students):
    #INPUT: pd Series containing the list of their usernames
    #OUTPUT: pd DataFrame with students organized by score and number of kata they have completed
    students = pd.DataFrame({
            "Name": [],
            "User": [],
            "Rank": [],
            "Score": [],
            "Completed": []
        })
    for i in list_of_students:
        new_data = readUserSummary(getSummaryResponse(i))
        students = pd.concat([students,new_data],ignore_index=True)
    return students.sort_values(by=['Score','Completed'],ascending=False)

## USER CHALLENGE FUNCTIONS

In [ ]:
def getChallengeResponse(user, page = 0):
    #INPUT: string of the student's username
    #OUTPUT: API Response from the codewars website, list of completed challenges
    
    response = requests.get(f"https://www.codewars.com/api/v1/users/{user}/code-challenges/completed?page={page}")
    
    return response

In [ ]:
def checkChallenge(username, assignmentid):
    #INPUT: string of the username, string of the assignment id
    #OUTPUT: Boolean on the result of the search
    
    kataPages = getChallengeResponse(username).json()["totalPages"]
    #print(type(kataPages))
    
    for page in range(kataPages):
        challengeList = getChallengeResponse(username, page).json()["data"]
        for kata in challengeList:
            if kata["id"] == assignmentid:
                return (kata["name"], kata['completedLanguages'], True)
    return (kataDetails(assignmentid)["Name"],kata['completedLanguages'], False)

## Bulk Read and Grade Functions

In [ ]:
def openGoogleSheet(sheet_id, sheet_name):
    pass
    #url = 

In [ ]:
def readUserCompleted(csvUserids, csvHomeworkids):
    #INPUT: 
        # username of the student,
        # csv of the list of ids for the homework
            # homework ids
            # homework names
            # homework points
    #OUTPUT: points gained / points listed * 100
    for user in csvUserids:
        for hw in csvHomeworkids:
            checkChallenge(user, hw)

## Loading the Google Sheet

each sheet corresponds to a different hw assignment and also a different HW set of questions


In [ ]:
#security, make sure noone else can find my personal google sheet

from configparser import ConfigParser

parser = ConfigParser()
_ = parser.read("notebook.cfg")

sheet_id = parser.get("sheet_id","auth_key")

In [ ]:
a_period = "A%20Roster"
h_period = "H%20Roster"


hw_1 = "hw1"
hw_2 = "hw2"
hw_3 = "hw3"

a_roster = pd.read_csv(f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={a_period}")
h_roster = pd.read_csv(f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={h_period}")

hw1_df = pd.read_csv(f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={hw_1}")
hw2_df = pd.read_csv(f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={hw_2}")
hw3_df = pd.read_csv(f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={hw_3}")

## Grading Function
* inputs: Roster Dataframe, Homework Dataframe
* outputs: Grading Dataframe: [columns: Name, rows: T/F Completed]
---

In [ ]:
def grading(roster, hw_df, missing=False):
    df_students = pd.DataFrame()  
    columns = ["First", "Last", "Username"]
    print(roster.loc[:,columns])
    for index in range(len(roster.loc[:,columns])):
        student = roster.loc[index,columns]
        user = student["Username"]
        name = student["First"]
        print(f"Grading: {name}")
        grades = []
        for question in hw_df["Kata ID"]:
            grade = checkChallenge(user, question)
            if missing:
                #print(f"    Missing:")
                if grade[2]==False:
                    print(f"    {grade[0].iloc[0]}")
            grades.append(grade[2])
        df_students.insert(0, ", ".join(student[["Last","First"]]), grades)
    df_students["Qs"] = hw_df["name"]
    print("Finished")
    return df_students.set_index("Qs")

#hw2_a = grading(a_roster,hw2_df)
#hw2_h = grading(h_roster,hw2_df)

In [ ]:
points = lambda graded_hw, points: graded_hw.sum().div(len(graded_hw)).mul(points).sort_index()


## HW 1 Grading


In [ ]:
hw1_a = grading(a_roster,hw1_df)
#time.sleep(5)
#hw1_h = grading(h_roster,hw1_df, True)

In [ ]:
print("A ROSTER HW 1")
print(points(hw1_a, 12))
#print("H ROSTER HW 1")
#print(points(hw1_h, 12))

In [ ]:
#hw1_a
hw1_h

## HW 2 Grading

In [ ]:
hw2_a = grading(a_roster,hw2_df,True)
#time.sleep(5)
#hw2_h = grading(h_roster,hw2_df,True)

In [ ]:
#print("A ROSTER HW 2")
#print(points(hw2_a, 12))
print("H ROSTER HW 2")
print(points(hw2_h, 12))

### checking hw2 for roster H

hw2_h = grading(h_roster,hw2_df, True)

In [ ]:
hw2_a.sort_index(1)
#hw2_h.sort_index(1)

## HW 3 Grading

In [ ]:
#hw3_a = grading(a_roster, hw3_df,True)
#time.sleep(5)
hw3_h = grading(h_roster,hw3_df,True)

In [ ]:
print("A ROSTER HW 3")
print(points(hw3_a, 11))
print("H ROSTER HW 3")
print(points(hw3_h, 11))